In [49]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [51]:
import torch 
import numpy as np

In [52]:
import tttorch as ttt

In [53]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [54]:
mat = ttt.random_matrix([[2, 2, 2, 2], [3, 3, 3, 3]])

In [55]:
print(mat)

A TT-Matrix of size 16 x 81, underlying tensorshape: [2, 2, 2, 2] x [3, 3, 3, 3], TT-ranks: [1, 2, 2, 2, 1] 
 on device 'cpu' 


In [56]:
mat.to(device)

In [57]:
print(mat)

A TT-Matrix of size 16 x 81, underlying tensorshape: [2, 2, 2, 2] x [3, 3, 3, 3], TT-ranks: [1, 2, 2, 2, 1] 
 on device 'cpu' 


In [58]:
mat.detach()

In [59]:
print(mat)

A TT-Matrix of size 16 x 81, underlying tensorshape: [2, 2, 2, 2] x [3, 3, 3, 3], TT-ranks: [1, 2, 2, 2, 1] 
 on device 'cpu' 


In [60]:
m = mat.full()

In [62]:
mat2 = ttt.to_tt_matrix(m, shape=[[2, 2, 2, 2], [3, 3, 3, 3]])

In [68]:
mat = ttt.random_matrix([[2, 2, 2, 2], [3, 3, 3, 3]], tt_rank=5)

In [89]:
mat.tt_cores[0].shape

torch.Size([1, 2, 3, 5])

In [69]:
for core in mat.tt_cores:
    print(core.shape)

torch.Size([1, 2, 3, 5])
torch.Size([5, 2, 3, 5])
torch.Size([5, 2, 3, 5])
torch.Size([5, 2, 3, 1])


In [76]:
def ind2sub(siz, idx):
    '''
    Translates full-format index into tt.vector one's.
    ----------
    Parameters:
        siz - tt.vector modes
        idx - full-vector index
    Note: not vectorized.
    '''
    n = len(siz)
    subs = np.empty((n))
    k = np.cumprod(siz[:-1])
    k = np.concatenate((np.ones(1), k))
    for i in range(n - 1, -1, -1):
        subs[i] = np.floor(idx / k[i])
        idx = idx % k[i]
    return subs.astype(np.int32)

In [116]:
def ind2sub_torch(siz, idx):
    n = len(siz)
    b = len(idx)
    subs = []
    k = np.cumprod(siz[:-1])
    k = np.concatenate((np.ones(1), k))
    

    for i in range(n - 1, -1, -1):
        subs.append(torch.floor(idx.float() / k[i]))
        idx = torch.fmod(idx, k[i])
        
        
    return torch.stack(subs, dim=1)
        
    
    

In [121]:
mat.raw_shape

[[2, 2, 2, 2], [3, 3, 3, 3]]

In [118]:
ind2sub_torch([2, 3, 2], torch.LongTensor([0, 1, 2, 3, 4, 5, 6, 7]))

tensor([[ 0.,  0.,  0.],
        [ 0.,  0.,  1.],
        [ 0.,  1.,  0.],
        [ 0.,  1.,  1.],
        [ 0.,  2.,  0.],
        [ 0.,  2.,  1.],
        [ 1.,  0.,  0.],
        [ 1.,  0.,  1.]])